In [1]:
import pandas as pd
data = pd.read_csv('E:\\elastic\\elasticsearch-5.4.3\\logs\\elasticsearch.log',sep='\r\n',encoding='utf-8')
data



e:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,"[2018-09-09T08:09:16,259][INFO ][o.e.n.Node ] [] initializing ..."
0,"[2018-09-09T08:09:16,510][INFO ][o.e.e.NodeEnv..."
1,"[2018-09-09T08:09:16,511][INFO ][o.e.e.NodeEnv..."
2,"[2018-09-09T08:09:16,861][INFO ][o.e.n.Node ..."
3,"[2018-09-09T08:09:16,862][INFO ][o.e.n.Node ..."
4,"[2018-09-09T08:09:16,863][INFO ][o.e.n.Node ..."
5,"[2018-09-09T08:09:19,426][INFO ][o.e.p.Plugins..."
6,"[2018-09-09T08:09:19,426][INFO ][o.e.p.Plugins..."
7,"[2018-09-09T08:09:19,427][INFO ][o.e.p.Plugins..."
8,"[2018-09-09T08:09:19,427][INFO ][o.e.p.Plugins..."
9,"[2018-09-09T08:09:19,427][INFO ][o.e.p.Plugins..."


In [ ]:
import nltk
import pandas as pd

data = pd.read_csv('E:\\elastic\\elasticsearch-5.4.3\\logs\\elasticsearch.log', sep='\r\n', encoding='utf-8')
data.columns = ['message']

stopwords = ['at']
special = ['[', ']']


def tokenizer(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in stopwords and token not in special]

    filtered_tokens = []
    for token in tokens:
        if nltk.re.search('\D[a-zA-Z]', token):
            filtered_tokens.append(token)
    return ' '.join(filtered_tokens)


filter_documents = data.apply(lambda text: tokenizer(text['message']), axis=1)
# print(list(s))

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
feature_matrix = vectorizer.fit_transform(list(filter_documents)).astype(float)

tfidf_features = feature_matrix
query_docs = ['sun nio ch IOUtil read IOUtil java',
              'INFO Node qXJqPwZ stopped'
              ]

query_docs_tfidf = vectorizer.transform(query_docs)

import numpy as np


def computer_cosine_similarity(doc_features, corpus_features, top_n=3):
    # get document vectors
    doc_features = doc_features.A[0]
    corpus_features = corpus_features.A
    # compute similarities
    similarity = np.dot(doc_features, corpus_features.T)
    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3)) for index in top_docs]
    return top_docs_with_score


for index, doc in enumerate(query_docs):
    doc_tfidf = query_docs_tfidf[index]
    top_similarity_docs = computer_cosine_similarity(doc_tfidf, tfidf_features, top_n=2)
    for doc_index, sim_score in top_similarity_docs:
        print(filter_documents[doc_index])
        print(sim_score)
